In [1]:
%pip install -q keras-nightly
%pip install scipy
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
fileName = "isFollow"
mangas_df = pd.read_csv('../Collaborative Filtering/mangas.csv')
behaviors_df = pd.read_csv(f'../Collaborative Filtering/behaviors_{fileName}.csv')

In [4]:
if fileName == "isFollow":
    behaviors_df['isFollow'] = behaviors_df['isFollow'].replace({True: 5, False: 0}).infer_objects(copy=False)
    # Hoặc chuyển sang hành vi tương lai
    pd.set_option('future.no_silent_downcasting', True)
    behaviors_df['isFollow'] = behaviors_df['isFollow'].replace({True: 5, False: 0})

C:\Users\Admin\AppData\Local\Temp\ipykernel_9600\3931985075.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  behaviors_df['isFollow'] = behaviors_df['isFollow'].replace({True: 5, False: 0}).infer_objects(copy=False)


In [5]:
behaviors_df[fileName].fillna(0, inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_9600\1190497169.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  behaviors_df[fileName].fillna(0, inplace=True)


In [6]:
user_id_list = behaviors_df['userId'].unique()
manga_id_list = mangas_df['_id'].unique()
expanded_data = []

In [7]:
for user_id in user_id_list:
    for manga_id in manga_id_list:
        expanded_data.append({'userId': user_id, '_id': manga_id})

In [8]:
expanded_df = pd.DataFrame(expanded_data)


In [9]:
expanded_df.rename(columns={'_id': 'mangaId'}, inplace=True)
merged_df = pd.merge(expanded_df, behaviors_df, on=['userId', 'mangaId'], how='left')

In [10]:
merged_df[fileName] = merged_df[fileName].fillna(0)
merged_df.drop(columns=['updatedAt'], inplace=True)
merged_df

,userId,mangaId,isFollow
0,5f8923f3948be104b0830fd1,662a9e73d2a9173d53c01b9d,5.0
1,5f8923f3948be104b0830fd1,662a9e74d2a9173d53c01b9e,0.0
2,5f8923f3948be104b0830fd1,662a9e75d2a9173d53c01b9f,0.0
3,5f8923f3948be104b0830fd1,662a9e76d2a9173d53c01ba0,0.0
4,5f8923f3948be104b0830fd1,662a9e77d2a9173d53c01ba1,0.0
...,...,...,...
745,5f892410948be104b0830fee,662a9e87d2a9173d53c01bb1,0.0
746,5f892410948be104b0830fee,662a9e88d2a9173d53c01bb2,0.0
747,5f892410948be104b0830fee,662a9e89d2a9173d53c01bb3,0.0
748,5f892410948be104b0830fee,662a9e8ad2a9173d53c01bb4,0.0


In [11]:
mangas_df.rename(columns={'_id': 'mangaId'}, inplace=True)
datas = pd.merge(mangas_df, merged_df, on='mangaId').drop(['author','genre'],axis=1)

datas.head()

,mangaId,name,userId,isFollow
0,662a9e73d2a9173d53c01b9d,One Piece,5f8923f3948be104b0830fd1,5.0
1,662a9e73d2a9173d53c01b9d,One Piece,5f8923f4948be104b0830fd2,0.0
2,662a9e73d2a9173d53c01b9d,One Piece,5f8923f5948be104b0830fd3,0.0
3,662a9e73d2a9173d53c01b9d,One Piece,5f8923f6948be104b0830fd4,0.0
4,662a9e73d2a9173d53c01b9d,One Piece,5f8923f7948be104b0830fd5,0.0


In [12]:
userDatas = datas.pivot_table(index=['userId'],columns=['mangaId'],values=fileName)
userDatas.head()
# print("Before: ",userViews.shape)
# userViews = userViews.dropna(thresh=10, axis=1).fillna(0,axis=1)
# #userViews.fillna(0, inplace=True)
# print("After: ",userViews.shape)

mangaId,662a9e73d2a9173d53c01b9d,662a9e74d2a9173d53c01b9e,662a9e75d2a9173d53c01b9f,662a9e76d2a9173d53c01ba0,662a9e77d2a9173d53c01ba1,662a9e78d2a9173d53c01ba2,662a9e79d2a9173d53c01ba3,662a9e7ad2a9173d53c01ba4,662a9e7bd2a9173d53c01ba5,662a9e7cd2a9173d53c01ba6,...,662a9e82d2a9173d53c01bac,662a9e83d2a9173d53c01bad,662a9e84d2a9173d53c01bae,662a9e85d2a9173d53c01baf,662a9e86d2a9173d53c01bb0,662a9e87d2a9173d53c01bb1,662a9e88d2a9173d53c01bb2,662a9e89d2a9173d53c01bb3,662a9e8ad2a9173d53c01bb4,662a9e8bd2a9173d53c01bb5
userId,,,,,,,,,,,,,,,,,,,,,
5f8923f3948be104b0830fd1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5f8923f4948be104b0830fd2,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5f8923f5948be104b0830fd3,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5f8923f6948be104b0830fd4,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5f8923f7948be104b0830fd5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
def standardize(row):
    new_row = (row - row.mean())/(row.max()-row.min())
    return new_row

df_std = userDatas.apply(standardize).T
df_std.fillna(0, inplace=True)
sparse_df = sparse.csr_matrix(df_std.values)
corrMatrix = pd.DataFrame(cosine_similarity(sparse_df),index=userDatas.columns,columns=userDatas.columns)
corrMatrix

mangaId,662a9e73d2a9173d53c01b9d,662a9e74d2a9173d53c01b9e,662a9e75d2a9173d53c01b9f,662a9e76d2a9173d53c01ba0,662a9e77d2a9173d53c01ba1,662a9e78d2a9173d53c01ba2,662a9e79d2a9173d53c01ba3,662a9e7ad2a9173d53c01ba4,662a9e7bd2a9173d53c01ba5,662a9e7cd2a9173d53c01ba6,...,662a9e82d2a9173d53c01bac,662a9e83d2a9173d53c01bad,662a9e84d2a9173d53c01bae,662a9e85d2a9173d53c01baf,662a9e86d2a9173d53c01bb0,662a9e87d2a9173d53c01bb1,662a9e88d2a9173d53c01bb2,662a9e89d2a9173d53c01bb3,662a9e8ad2a9173d53c01bb4,662a9e8bd2a9173d53c01bb5
mangaId,,,,,,,,,,,,,,,,,,,,,
662a9e73d2a9173d53c01b9d,1.000000,0.0,-0.061898,-0.049629,-0.049629,0.0,-0.049629,-0.049629,-0.049629,-0.034483,...,0.0,-0.034483,-0.034483,-0.049629,-0.034483,-0.034483,0.0,-0.049629,-0.034483,-0.034483
662a9e74d2a9173d53c01b9e,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
662a9e75d2a9173d53c01b9f,-0.061898,0.0,1.000000,-0.089087,-0.089087,0.0,-0.089087,-0.089087,-0.089087,-0.061898,...,0.0,-0.061898,-0.061898,-0.089087,-0.061898,-0.061898,0.0,-0.089087,-0.061898,-0.061898
662a9e76d2a9173d53c01ba0,-0.049629,0.0,-0.089087,1.000000,0.464286,0.0,-0.071429,-0.071429,-0.071429,-0.049629,...,0.0,-0.049629,-0.049629,-0.071429,-0.049629,-0.049629,0.0,-0.071429,-0.049629,-0.049629
662a9e77d2a9173d53c01ba1,-0.049629,0.0,-0.089087,0.464286,1.000000,0.0,-0.071429,-0.071429,-0.071429,-0.049629,...,0.0,-0.049629,-0.049629,-0.071429,-0.049629,-0.049629,0.0,-0.071429,-0.049629,-0.049629
662a9e78d2a9173d53c01ba2,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
662a9e79d2a9173d53c01ba3,-0.049629,0.0,-0.089087,-0.071429,-0.071429,0.0,1.000000,0.464286,-0.071429,-0.049629,...,0.0,-0.049629,-0.049629,-0.071429,-0.049629,-0.049629,0.0,-0.071429,-0.049629,-0.049629
662a9e7ad2a9173d53c01ba4,-0.049629,0.0,-0.089087,-0.071429,-0.071429,0.0,0.464286,1.000000,0.464286,-0.049629,...,0.0,-0.049629,-0.049629,-0.071429,-0.049629,-0.049629,0.0,-0.071429,-0.049629,-0.049629
662a9e7bd2a9173d53c01ba5,-0.049629,0.0,-0.089087,-0.071429,-0.071429,0.0,-0.071429,0.464286,1.000000,-0.049629,...,0.0,-0.049629,-0.049629,-0.071429,-0.049629,-0.049629,0.0,-0.071429,-0.049629,-0.049629


In [14]:
corrMatrix = userDatas.corr(method='pearson')
corrMatrix.head(6)

mangaId,662a9e73d2a9173d53c01b9d,662a9e74d2a9173d53c01b9e,662a9e75d2a9173d53c01b9f,662a9e76d2a9173d53c01ba0,662a9e77d2a9173d53c01ba1,662a9e78d2a9173d53c01ba2,662a9e79d2a9173d53c01ba3,662a9e7ad2a9173d53c01ba4,662a9e7bd2a9173d53c01ba5,662a9e7cd2a9173d53c01ba6,...,662a9e82d2a9173d53c01bac,662a9e83d2a9173d53c01bad,662a9e84d2a9173d53c01bae,662a9e85d2a9173d53c01baf,662a9e86d2a9173d53c01bb0,662a9e87d2a9173d53c01bb1,662a9e88d2a9173d53c01bb2,662a9e89d2a9173d53c01bb3,662a9e8ad2a9173d53c01bb4,662a9e8bd2a9173d53c01bb5
mangaId,,,,,,,,,,,,,,,,,,,,,
662a9e73d2a9173d53c01b9d,1.000000,NaN,-0.061898,-0.049629,-0.049629,NaN,-0.049629,-0.049629,-0.049629,-0.034483,...,NaN,-0.034483,-0.034483,-0.049629,-0.034483,-0.034483,NaN,-0.049629,-0.034483,-0.034483
662a9e74d2a9173d53c01b9e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
662a9e75d2a9173d53c01b9f,-0.061898,NaN,1.000000,-0.089087,-0.089087,NaN,-0.089087,-0.089087,-0.089087,-0.061898,...,NaN,-0.061898,-0.061898,-0.089087,-0.061898,-0.061898,NaN,-0.089087,-0.061898,-0.061898
662a9e76d2a9173d53c01ba0,-0.049629,NaN,-0.089087,1.000000,0.464286,NaN,-0.071429,-0.071429,-0.071429,-0.049629,...,NaN,-0.049629,-0.049629,-0.071429,-0.049629,-0.049629,NaN,-0.071429,-0.049629,-0.049629
662a9e77d2a9173d53c01ba1,-0.049629,NaN,-0.089087,0.464286,1.000000,NaN,-0.071429,-0.071429,-0.071429,-0.049629,...,NaN,-0.049629,-0.049629,-0.071429,-0.049629,-0.049629,NaN,-0.071429,-0.049629,-0.049629
662a9e78d2a9173d53c01ba2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
def get_recommendations(user_id, userViews, corrMatrix, num_recommendations=36):
    # Lấy danh sách các mục mà người dùng đã xem
    user_data = userViews.loc[user_id]
    watched_items = user_data[user_data > 0].index.tolist()

    # Tạo một dictionary để lưu điểm số gợi ý
    recommendations = {}

    # Duyệt qua các mục mà người dùng đã xem
    for item in watched_items:
        similar_items = corrMatrix[item].sort_values(ascending=False)

        for similar_item, score in similar_items.items():
            if similar_item in recommendations:
                recommendations[similar_item] += score
            else:
                recommendations[similar_item] = score

        # Bao gồm cả bộ truyện hiện tại mà người dùng đã xem
        if item in recommendations:
            recommendations[item] += 1  # Hoặc điểm số nào đó bạn thấy phù hợp
        else:
            recommendations[item] = 1  # Hoặc điểm số nào đó bạn thấy phù hợp

    # Sắp xếp các gợi ý theo điểm số
    sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)

    # Trả về top N gợi ý
    return sorted_recommendations[:num_recommendations]

# Ví dụ sử dụng hàm get_recommendations
user_id = '5f892400948be104b0830fde'
recommendations = get_recommendations(user_id, userDatas, corrMatrix)
recommendations


[('662a9e75d2a9173d53c01b9f', 2.0),
 ('662a9e73d2a9173d53c01b9d', -0.061898446059017204),
 ('662a9e7cd2a9173d53c01ba6', -0.061898446059017245),
 ('662a9e87d2a9173d53c01bb1', -0.061898446059017245),
 ('662a9e84d2a9173d53c01bae', -0.061898446059017245),
 ('662a9e81d2a9173d53c01bab', -0.061898446059017245),
 ('662a9e8bd2a9173d53c01bb5', -0.061898446059017245),
 ('662a9e80d2a9173d53c01baa', -0.06189844605901726),
 ('662a9e7ed2a9173d53c01ba8', -0.061898446059017266),
 ('662a9e83d2a9173d53c01bad', -0.06189844605901728),
 ('662a9e86d2a9173d53c01bb0', -0.06189844605901728),
 ('662a9e8ad2a9173d53c01bb4', -0.06189844605901729),
 ('662a9e7dd2a9173d53c01ba7', -0.08908708063747472),
 ('662a9e77d2a9173d53c01ba1', -0.08908708063747472),
 ('662a9e7ad2a9173d53c01ba4', -0.08908708063747475),
 ('662a9e79d2a9173d53c01ba3', -0.08908708063747475),
 ('662a9e85d2a9173d53c01baf', -0.08908708063747475),
 ('662a9e76d2a9173d53c01ba0', -0.08908708063747475),
 ('662a9e89d2a9173d53c01bb3', -0.08908708063747475),
 ('

In [16]:
import math

# Lấy danh sách các ID mà không bao gồm những giá trị là nan
filtered_recommendations = [item_id for item_id, score in recommendations if not math.isnan(score)]

filtered_recommendations

['662a9e75d2a9173d53c01b9f',
 '662a9e73d2a9173d53c01b9d',
 '662a9e7cd2a9173d53c01ba6',
 '662a9e87d2a9173d53c01bb1',
 '662a9e84d2a9173d53c01bae',
 '662a9e81d2a9173d53c01bab',
 '662a9e8bd2a9173d53c01bb5',
 '662a9e80d2a9173d53c01baa',
 '662a9e7ed2a9173d53c01ba8',
 '662a9e83d2a9173d53c01bad',
 '662a9e86d2a9173d53c01bb0',
 '662a9e8ad2a9173d53c01bb4',
 '662a9e7dd2a9173d53c01ba7',
 '662a9e77d2a9173d53c01ba1',
 '662a9e7ad2a9173d53c01ba4',
 '662a9e79d2a9173d53c01ba3',
 '662a9e85d2a9173d53c01baf',
 '662a9e76d2a9173d53c01ba0',
 '662a9e89d2a9173d53c01bb3',
 '662a9e7fd2a9173d53c01ba9',
 '662a9e7bd2a9173d53c01ba5']